In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

In [2]:
epochs = 50
batch_size = 2
n_qbits = 9
num_train_images = 10

quantum_device = qml.device("default.qubit", wires=n_qbits)

In [3]:
"""
PLOTTING FUNCTIONS
"""

def plot_loss_metric(loss, val_loss, metric, val_metric, metric_name='accuracy'):

    epochs_range = range(len(loss))

    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    if metric_name == 'accuracy':
        plt.plot(epochs_range, metric, label='Training Accuracy')
        plt.plot(epochs_range, val_metric, label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
    elif metric_name == 'recall':
        plt.plot(epochs_range, metric, label='Training Recall')
        plt.plot(epochs_range, val_metric, label='Validation Recall')
        plt.title('Training and Validation Recall')
    elif metric_name == 'precision':
        plt.plot(epochs_range, metric, label='Training Precision')
        plt.plot(epochs_range, val_metric, label='Validation Precision')
        plt.title('Training and Validation Precision')
    plt.legend(loc='lower right')
    

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

In [4]:
def prep_dataset(dataset, labels, val_split=0.25):

    dataset = dataset/255.
    dataset = np.expand_dims(dataset, axis=3)
    x_train, x_val, y_train, y_val = train_test_split(dataset, labels, test_size=0.25, random_state=25)
    y_train = keras.utils.to_categorical(y_train)
    y_val = keras.utils.to_categorical(y_val)

    return x_train, x_val, y_train, y_val

Classical custom conv, problems with the outputs being (28,28,1) and not (None, 28, 28, 1) as usual shown in model.summary()

In [37]:
def conv(input, weight):

    output = tf.math.multiply(input, weight)
    output = tf.math.reduce_mean(output)
        
    return output


class CustomConv(keras.layers.Layer):
    def __init__(self, dim_kernel=3, filters=8):
        
        super().__init__()
        self.dim_kernel = dim_kernel
        self.filters = filters

    
    def build(self, input_shape):

        self.w = self.add_weight(
            shape=(self.dim_kernel, self.dim_kernel, self.filters),
            initializer='uniform',
            trainable=True
        )
        super().build(input_shape)


    def call(self, inputs):

        output_tensor = []
        padded_image = tf.pad(inputs, paddings=[(0,0), (1,1), (1,1), (0,0)])

        for filter in range(self.filters):
            for i in range(28):
                for j in range(28):

                    temp_list = []
                    for k in range(inputs.shape[-1]):

                        input_tensor = padded_image[i:i+self.dim_kernel, j:j+self.dim_kernel, k]
                        temp_list.append(self.conv(input_tensor, self.w[:,:,filter]))
                    
                    temp_list = tf.math.reduce_mean(temp_list)
                    output_tensor.append(tf.keras.activations.relu(temp_list))
        
        output_tensor = tf.stack(output_tensor)
        output_tensor = tf.reshape(output_tensor, shape=(-1, 28, 28, self.filters))

        return output_tensor

In [38]:
model = Sequential([
    layers.InputLayer(input_shape=(28,28,1), batch_size=batch_size),
    CustomConv(),
    #layers.Conv2D(16, 3, trainable=False, padding='same'),
    #layers.Conv2D(16, 3, trainable=False, padding='same'),
    layers.Flatten(),
    layers.Dense(10, activation='softmax', trainable=False)
])

loss = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['accuracy']
)

model.build()
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 custom_conv_21 (CustomConv)  (1, 28, 28, 8)           72        
                                                                 
 flatten_1 (Flatten)         (1, 6272)                 0         
                                                                 
 dense_1 (Dense)             (1, 10)                   62730     
                                                                 
Total params: 62,802
Trainable params: 72
Non-trainable params: 62,730
_________________________________________________________________


In [39]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_images = train_images[:num_train_images]
train_labels = train_labels[:num_train_images]

x_train, x_val, y_train, y_val = prep_dataset(train_images, train_labels)

The train doesn't even start

In [40]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size=batch_size,
    validation_data=(x_val, y_val),
    epochs=epochs,
    shuffle=True,
)

Epoch 1/50


Quantum custom conv based on the previous, the only difference is the function that outputs the feature maps, a 
convolution for the former cells and a quantum circuit for the subsequent cells

In [196]:
@qml.qnode(quantum_device, interface="tf")
def q_circuit(input, weights):
            
    features = tf.squeeze(input)
    num_qbits = len(features)

    qml.AngleEmbedding(features=features, wires=range(n_qbits), rotation='X')
    qml.BasicEntanglerLayers(weights=weights, wires=range(n_qbits))
            
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qbits)]


class QuantumConvolution(keras.layers.Layer):

    def __init__(self, q_filters=16, dim_q_kernel=3, circuit_layers=2):

        super().__init__()
        self.dim_q_kernel = dim_q_kernel
        self.q_filters = q_filters
        self.circuit_layers = circuit_layers
        self.w = self.add_weight(
            shape = (self.circuit_layers, (self.dim_q_kernel)**2, self.q_filters),
            initializer = 'random_normal',
            trainable=True
        )

    def build(self, input_shape):

        self.rows = input_shape[0]
        self.cols = input_shape[1]
        self.channels_in = input_shape[2]

    def call(self, inputs):

        output_tensor = []
        padded_image = tf.pad(inputs, paddings=[(1,1), (1,1), (0,0)])

        for filter in range(self.q_filters):
            for i in range(self.rows):
                for j in range(self.cols):

                    temp_list = []
                    for k in range(self.channels_in):

                        input_tensor = padded_image[i:i+self.dim_q_kernel, j:j+self.dim_q_kernel, k]
                        measurements = q_circuit(input_tensor, self.w[:,:,filter])
                        measurements = tf.math.reduce_mean(measurements)
                        temp_list.append(measurements)
                    
                    temp_list = tf.math.reduce_mean(temp_list)
                    output_tensor.append(tf.keras.activations.relu(temp_list))
        
        output_tensor = tf.stack(output_tensor)
        output_tensor = tf.reshape(output_tensor, shape=(self.rows, self.cols, self.q_filters))

        return output_tensor

In [197]:
model = Sequential([
    QuantumConvolution(),
    #layers.Conv2D(16,3, trainable=False),
])

loss = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['accuracy']
)

model.build(input_shape=(28,28,1))
model.summary()

Attempt to put a quantum layer as a dense layer at the end of the classical model, used the pennylan tool
for converting a quantum function to a keras layer (https://pennylane.ai/qml/demos/tutorial_qnn_module_tf.html)
it presents a bug as it states that the weights of the quantum circuit are 'unused' as one can see
in model.summary()

In [3]:
@qml.qnode(quantum_device)
def quantum_convolution(
    inputs,
    weights):

    features = np.ravel(inputs)
    qml.AngleEmbedding(features, wires=range(n_qbits), rotation='X')
    qml.BasicEntanglerLayers(weights, wires=range(n_qbits))

    return [qml.expval(qml.PauliZ(j)) for j in range(n_qbits)]

In [4]:
weight_shapes = {"weights": (2, 9)}

qlayer = qml.qnn.KerasLayer(quantum_convolution, weight_shapes, output_dim=n_qbits)


In [5]:
model = Sequential([
    layers.Conv2D(1,3, activation='relu', padding='same', trainable=False, input_shape=(28,28,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(1,3, activation='relu', padding='same', trainable=False),
    layers.MaxPooling2D(),
    layers.Conv2D(1,3, activation='relu', padding='same', trainable=False),
    layers.MaxPooling2D(),
    qlayer,
    layers.Dense(10, activation='softmax', trainable=False)
])

loss = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 1)         10        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 1)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 1)         10        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 1)          0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 1)           10        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 1)          0

In [8]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_images = train_images[:num_train_images]
train_labels = train_labels[:num_train_images]

x_train, x_val, y_train, y_val = prep_dataset(train_images, train_labels)

Although the 'unused' next to the quantum circuit layer in the previous model.summry() the weights seems to enter in the
optimization routine as all the other layers are set to 'trainable = False' and the loss stil goes down 

In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size=batch_size,
    validation_data=(x_val, y_val),
    epochs=epochs,
    shuffle=True,
)